### Navigation 
1. [Day 21](#Day-21)  
1. [Day 24](#Day-24)  
1. [Day 25](#Day-25)  

# Day 21
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/21

### --- Part One ---

In [1]:
inputs = """Player 1 starting position: 4
Player 2 starting position: 1"""
# inputs = """Player 1 starting position: 4
# Player 2 starting position: 8"""
inputs = inputs.split('\n')
p1_start = int(inputs[0][-1])
p2_start = int(inputs[1][-1])
p1_start, p2_start

(4, 1)

In [2]:
class DeterministicDice:
    def __init__(self):
        self.last_value = 0
        self.roll_count = 0
        
    def roll(self):
        if self.last_value == 100:
            self.last_value = 0
        self.roll_count += 1
        self.last_value += 1
        return self.last_value
    
    def roll_three(self):
        roll_sum = sum([self.roll() for i in range(3)])
        return roll_sum

class Player:
    def __init__(self, name, start_pos, max_score=1000):
        self.score = 0
        self.pos = start_pos
        self.name = name
        self.win = False
        self.max_score = max_score
        
    def move(self, roll_value):
        self.pos = (self.pos + roll_value) % 10
        if self.pos == 0:
            self.score += 10
        else:
            self.score += self.pos
        if self.score >= self.max_score:
            self.win = True

In [3]:
def game_loop(p: Player, dice: DeterministicDice):
    roll_v = dice.roll_three()
    p.move(roll_v)
    return p, dice

In [4]:
dice = DeterministicDice()
p1 = Player('p1', p1_start)
p2 = Player('p2', p2_start)

while True:
    p1, dice = game_loop(p1, dice)
    if p1.win:
        print("Game over, score:", p2.score * dice.roll_count)
        break
    p2, dice = game_loop(p2, dice)
    if p2.win:
        print("Game over, score:", p1.score * dice.roll_count)
        break

Game over, score: 913560


### --- Part Two ---

# Day 24
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/24

In [5]:
with open("data/day24-input.txt", 'r') as file:
    inputs = file.read()#.replace('w', 'w0')
    inputs = inputs.split('\n')

In [6]:
# ALU operations

def add(x, y):
    if type(x) is int and type(y) is int:
        return x + y
    if x == 0:
        return y
    if y == 0:
        return x
    return '(%s+%s)' % (x, y)

assert(add(2,2) == 4)
assert(add(2,'w') == '(2+w)')
assert(add(2,'(w+1)') == '(2+(w+1))')
assert(add(0,'w') == 'w')
assert(add('w','w') == '(w+w)')
    
def mul(x, y):
    if type(x) is int and type(y) is int:
        return x * y
    if x == 0 or y == 0:
        return 0
    if y == 1:
        return x
    if x == 1:
        return y
    if ((type(x) is str and 'z' in x) or 
        (type(y) is str and 'z' in y)):
        return '(%s*%s)' % (x, y)
    if type(x) is str and type(y) is int:
        x = x.replace('(','').replace(')', '')
        xx = ''.join(['%s*%s+' % (y, a) for a in x.split('+')])
        return xx[:-1]

assert(mul(2,2) == 4)
# assert(mul(2,'w') == '(2*w)')
assert(mul(0,2) == 0)
assert(mul(0,'(w+1)') == 0)
assert(mul(1,'w') == 'w')
assert(mul('w',1) == 'w')
assert(mul('w',0) == 0)
# assert(mul('w','w') == '(w*w)')
assert(mul('w',26) == '26*w')
assert(mul('(w+4)',26) == '26*w+26*4')
    
def div(x, y):
    if type(x) is int and type(y) is int:
        return int(x / y)
    if x == 0:
        return 0
    if y == 1:
        return x
    if ((type(x) is str and 'z' in x) or 
        (type(y) is str and 'z' in y)):
        return 'int(%s/%s)' % (x, y)
    f = str(y)+'*'
    # divide by 'y'
    # if number contains a prefix of 'y*' remove prefix and keep result
    # otherwise:
    # since divident is always < y; w = <1,9>
    # division result will always be < 1
    # because we floor the results, anything < becomes 0
    result = '+'.join([n[:i] + n[i+len(f):]
                       for n in x.split('+')
                       for i in [n.find(f)]
                       if f in n])
    return '0' if result == '' else result

assert(div(4,4) == 1)
assert(div(0,'(w+4)') == 0)
# assert(div('(w*4)', 'w') == 'int((w*4)/w)')
assert(div('w*4', 1) == 'w*4')
assert(div('26*w+4', 26) == 'w')
assert(div('26*26*w+26*4', 26) == '26*w+4')
    
def mod(x, y):
    if type(x) is int and type(y) is int:
        return x % y
    if x == 0 or y == 1:
        return 0
    if type(x) is str and 'z' in x:
        return '(%s*%s)' % (x, y)
    f = str(y)+'*'
    return '+'.join([n for n in x.split('+')
                     if f not in n])

assert(mod(4,4) == 0)
assert(mod(4,3) == 1)
assert(mod(0,'(w+4)') == 0)
assert(mod('(w*4)', 1) == 0)

def eql(x, y):
    if type(x) is int and type(y) is int:
        return 1 if x == y else 0
    if x == y:
        return 1
    if type(x) is int and x not in range(1,10):
        return 0
    return '(1 if %s == %s else 0)' % (x, y)

assert(eql(4,4) == 1)
assert(eql(4,3) == 0)
assert(eql(0,'(w+4)') == 0)
assert(eql('(w*4)', 1) == '(1 if (w*4) == 1 else 0)')
assert(eql('(w*4)', '(w*4)') == 1)

op_dict = {
    'add': lambda x, y: add(x, y),
    'mul': lambda x, y: mul(x, y),
    'div': lambda x, y: div(x, y),
    'mod': lambda x, y: mod(x, y),
    'eql': lambda x, y: eql(x, y),
}

In [7]:
def create_var_dict(count):
    return {
        'x': 0,
        'y': 0,
        'z': 0 if count == 1 else 'z' + str(count - 1),
        'w': 'w' + str(count)
    }

w_count = 0
var_dict_list = []

for ins_str in inputs[:]:
    ins = ins_str.split(' ')
    op = ins[0]
    if op == 'inp':
        w_count += 1
        var_dict = create_var_dict(w_count)
        var_dict_list.append(var_dict)
    else:
        ins = ins_str.split(' ')
        op = ins[0]
        a = var_dict[ins[1]]
        b = var_dict[ins[2]] if ins[2] in var_dict.keys() else int(ins[2])
        var_dict[ins[1]] = op_dict[op](a, b)

In [8]:
for i, dic in enumerate(var_dict_list[:]):
    print('z'+str(i+1)+':', dic['z'])

z1: (w1+4)
z2: ((z1*((25*(1 if (1 if ((z1*26)+14) == w2 else 0) == 0 else 0))+1))+((w2+16)*(1 if (1 if ((z1*26)+14) == w2 else 0) == 0 else 0)))
z3: ((z2*((25*(1 if (1 if ((z2*26)+11) == w3 else 0) == 0 else 0))+1))+((w3+14)*(1 if (1 if ((z2*26)+11) == w3 else 0) == 0 else 0)))
z4: ((int(z3/26)*((25*(1 if (1 if ((z3*26)+-13) == w4 else 0) == 0 else 0))+1))+((w4+3)*(1 if (1 if ((z3*26)+-13) == w4 else 0) == 0 else 0)))
z5: ((z4*((25*(1 if (1 if ((z4*26)+14) == w5 else 0) == 0 else 0))+1))+((w5+11)*(1 if (1 if ((z4*26)+14) == w5 else 0) == 0 else 0)))
z6: ((z5*((25*(1 if (1 if ((z5*26)+15) == w6 else 0) == 0 else 0))+1))+((w6+13)*(1 if (1 if ((z5*26)+15) == w6 else 0) == 0 else 0)))
z7: ((int(z6/26)*((25*(1 if (1 if ((z6*26)+-7) == w7 else 0) == 0 else 0))+1))+((w7+11)*(1 if (1 if ((z6*26)+-7) == w7 else 0) == 0 else 0)))
z8: ((z7*((25*(1 if (1 if ((z7*26)+10) == w8 else 0) == 0 else 0))+1))+((w8+7)*(1 if (1 if ((z7*26)+10) == w8 else 0) == 0 else 0)))
z9: ((int(z8/26)*((25*(1 if (1 if (

Which can be reduced to the following:  
```python
z1: (w1+4)
z2: 26*w1+26*4+w2+16
z3: 26*26*w1+26*26*4+26*w2+26*16+w3+14
w4 = w3 + 14 - 13 = w3 + 1
z4: 26*w1+26*4+w2+16
z5: 26*26*w1+26*26*4+26*w2+26*16+w5+11
z6: 26*26*26*w1+26*26*26*4+26*26*w2+26*26*16+26*w5+26*11+w6+13
w7 = w6 + 13 - 7 = w6 + 6
z7: 26*26*w1+26*26*4+26*w2+26*16+w5+11
z8: 26*26*26*w1+26*26*26*4+26*26*w2+26*26*16+26*w5+26*11+w8+7
w9 = w8 + 7 - 12 = w8 - 5
z9: 26*26*w1+26*26*4+26*w2+26*16+w5+11
z10: 26*26*26*w1+26*26*26*4+26*26*w2+26*26*16+26*w5+26*11+w10+15
w11 = w10 + 15 - 16 = w10 - 1
z11: 26*26*w1+26*26*4+26*w2+26*16+w5+11
w12 = w5 + 11 - 9 = w5 + 2
z12: 26*w1+26*4+w2+16
w13 = w2 + 16 - 8 = w2 + 8
z12: w1 + 4
w14 = w1 + 4 - 8 = w1 - 4
z = 0
```

And that's how we extract the relationship between different 'w':
```python
w14 = w1 - 4
w13 = w2 + 8
w12 = w5 + 2
w11 = w10 - 1
w9 = w8 - 5
w7 = w6 + 6
w4 = w3 + 1
```

Above equations can be simplified to these 2 rules:
```python
if "div z 1":
    z[n] = 26*(z[n-1]) + w[n] + add_y[n]  
if "div z 26":
    w[n] = z[n-1] % 26 + add_x[n]
    z[n] = int(z[n-1]/26)
```

Now let's follow the same reduction process but with code (I had to modify my ALU functions as well)

In [9]:
instructions2 = [[inputs[j] for j in range(i, len(inputs), 18)] for i in range(0,18)]
w_dict = {}
z_dict = {-1:0}
add_y_list = [int(ins.split(' ')[-1]) for ins in instructions2[-3]]
add_x_list = [int(ins.split(' ')[-1]) for ins in instructions2[5]]
div_z_list = [int(ins.split(' ')[-1]) for ins in instructions2[4]]

In [10]:
for n, div_z in zip(range(0,14), div_z_list):
    if div_z == 1:
        # if "div z 1":
        #    z[n] = 26*(z[n-1]) + w[n] + add_y[n]
        z_dict[n] = add(
            mul(z_dict[n-1], 26),
            add('w'+str(n), add_y_list[n])
        )
    elif div_z == 26:
        # if "div z 26":
        #    w[n] = z[n-1] % 26 + add_x[n]
        #    z[n] = int(z[n-1]/26)
        w_dict['w'+str(n)] = add(
            mod(z_dict[n-1], 26),
            add_x_list[n]
        ).replace('+-', '-').replace('(', '').replace(')', '')
        z_dict[n] = div(z_dict[n-1], 26)
    z_dict[n] = z_dict[n].replace('(', '').replace(')', '')

print('W values pair relations:', w_dict)

print('\n\nZ values:')
z_dict

W values pair relations: {'w3': 'w2+14-13', 'w6': 'w5+13-7', 'w8': 'w7+7-12', 'w10': 'w9+15-16', 'w11': 'w4+11-9', 'w12': 'w1+16-8', 'w13': 'w0+4-8'}


Z values:


{-1: 0,
 0: 'w0+4',
 1: '26*w0+26*4+w1+16',
 2: '26*26*w0+26*26*4+26*w1+26*16+w2+14',
 3: '26*w0+26*4+w1+16',
 4: '26*26*w0+26*26*4+26*w1+26*16+w4+11',
 5: '26*26*26*w0+26*26*26*4+26*26*w1+26*26*16+26*w4+26*11+w5+13',
 6: '26*26*w0+26*26*4+26*w1+26*16+w4+11',
 7: '26*26*26*w0+26*26*26*4+26*26*w1+26*26*16+26*w4+26*11+w7+7',
 8: '26*26*w0+26*26*4+26*w1+26*16+w4+11',
 9: '26*26*26*w0+26*26*26*4+26*26*w1+26*26*16+26*w4+26*11+w9+15',
 10: '26*26*w0+26*26*4+26*w1+26*16+w4+11',
 11: '26*w0+26*4+w1+16',
 12: 'w0+4',
 13: '0'}

In [11]:
def optimize(k, v, w_val_dict, opt='max'):
    assert(opt in ['min', 'max'])
    w_range = range(1,10) # <1,9>
    if opt == 'max':
        w_range = list(reversed(w_range))
    wa = k
    wb = v.split('+')[0]
    for w in w_range:
        v_temp = v.replace(wb, str(w))
        res = eval(v_temp)
        if res in w_range:
            w_val_dict[wa] = res
            w_val_dict[wb] = w
            return w_val_dict

# maximize
w_val_dict = {}
for k, v in w_dict.items():
    w_val_dict = optimize(k,v, w_val_dict, opt='max')

''.join([str(w_val_dict['w'+str(i)]) for i in range(14)])

'91897399498995'

### --- Part Two ---

In [12]:
# now minimize
w_val_dict = {}
for k, v in w_dict.items():
    w_val_dict = optimize(k,v, w_val_dict, opt='min')

''.join([str(w_val_dict['w'+str(i)]) for i in range(14)])

'51121176121391'

In [13]:
def check_number(num_str, inputs):
    var_dict = {
        'x' : 0, 'y': 0, 'z': 0, 'w': 0
    }
    w_count = 0
    for ins_str in inputs[:]:
        ins = ins_str.split(' ')
        op = ins[0]
        if op == 'inp':
            var_dict['w'] = int(num_str[w_count])
            w_count += 1
        else:
            ins = ins_str.split(' ')
            op = ins[0]
            a = var_dict[ins[1]]
            b = var_dict[ins[2]] if ins[2] in var_dict.keys() else int(ins[2])
            var_dict[ins[1]] = op_dict[op](a, b)
    return var_dict


In [14]:
print(check_number('91897399498995', inputs)['z'])
print(check_number('51121176121391', inputs)['z'])

0
0


# Day 25
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/25

In [15]:
with open("data/day25-input.txt", 'r') as file:
    inputs = file.read()

In [16]:
import numpy as np


def move_cucumber(herd_id, inputs_map):
    x, y = np.where(inputs_map == herd_id)
    if herd_id == 1:
        y_target = np.where(y+1 < inputs_map.shape[1], y+1, 0)
        pos_move_mask = inputs_map[x, y_target] == 0
        x_target = x[pos_move_mask]
        y_target = y_target[pos_move_mask]
    elif herd_id == 2:
        x_target = np.where(x+1 < inputs_map.shape[0], x+1, 0)
        pos_move_mask = inputs_map[x_target, y] == 0
        x_target = x_target[pos_move_mask]
        y_target = y[pos_move_mask]
    
    x = x[pos_move_mask]
    y = y[pos_move_mask]
    # use pos_move_mask and move sea cucumbers
    inputs_map[x, y] = 0
    if herd_id == 1:
        inputs_map[x, y_target] = herd_id
    if herd_id == 2:
        inputs_map[x_target, y] = herd_id
    return inputs_map

In [17]:
sign_to_int = {
    'v': 2,
    '>': 1,
    '.': 0
}

inputs_map = [[sign_to_int[c] for c in line] for line in inputs.split('\n')]
inputs_map = np.asarray(inputs_map)
step_counter = 0
while True:
    # single step
    inputs_map_temp = inputs_map.copy()
    inputs_map_temp = move_cucumber(1, inputs_map_temp)
    inputs_map_temp = move_cucumber(2, inputs_map_temp)
    step_counter += 1
    if np.all(inputs_map == inputs_map_temp):
        break
    inputs_map = inputs_map_temp
    
print(step_counter)

278
